In [91]:
# Import Required Packages
import torch
import os
import json
import sys
import re
import random
import importlib.util
from typing import *
from tqdm import tqdm 
from typing import List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap


from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
from datasets import Dataset


from JS_Architects import *

# Summary

We will use Architects model

1) Implement Custom pipeline, no prompting just prediction with shrekking of the embedding 

2) Automated Forsequence Classification
    1) No prompting
    2) Prompt: "Classify with labels encode in binary"
    3) Prompt: "Clasify with list label: ["depth",...]"
    4) Prompt "Classify with list label: ["depth",...]. Depth represents..., Containment represents...."

# Custom Pipeline 

## 1. Preparation

### Load 


In [92]:
with open("perceptions_training.json", "r") as f:
    dic_training = json.load(f)

with open("perceptions_testing.json", "r") as f:
    dic_testing = json.load(f)

label_list = [
    "containment",
    "depth",
    "symmetry",
    "categorical",
    "spatial-Orientation",
    "spatial-Ordinal",
    "similarity",
    "quantitative",
    "replication",
    "figure-Ground",
    "continuity",
    "size",
    "closure",
    "centroid",
    "topological",
    "motion",
]

### Formatting

In [122]:
from datasets import Dataset

def prepare_data_for_multilabel_classification(
    dic_training,
    instruction,
    label_list,
    inp_prefix="I",
    out_prefix="O",
    arr_sep="\n",
    exa_sep="\n---\n",
    eos_token="<EOS>"
):
    llama_data = []

    # Create a mapping from label to index
    label_to_index = {label: i for i, label in enumerate(label_list)}

    for entry_id, content in dic_training.items():
        # Extract perceptions (labels) and encode them as a binary vector
        perceptions = content.get("perceptions", [])
        label_vector = labels_to_binary(label_list, perceptions)

        # Combine train and test examples
        examples = content.get("example", {}).get("train", []) + content.get("example", {}).get("test", [])

        # Format examples into a single input string
        formatted_examples = []
        for example in examples:
            input_data = f"{inp_prefix}{format_array(example['input'], arr_sep)}"
            output_data = f"{out_prefix}{format_array(example['output'], arr_sep)}{eos_token}"
            formatted_examples.append(f"{input_data}{exa_sep}{output_data}")

        # Combine all examples into one input text
        combined_text = exa_sep.join(formatted_examples)

        # Add the structured data for fine-tuning
        llama_data.append({
            "instruction": f"{instruction}",
            #"instruction": f"{instruction}: {', '.join(label_list)}.",
            "input": combined_text,
            "output": label_vector,  # Multi-label as binary vector
        })

    return llama_data

def format_array(array, arr_sep="\n"):
    """
    Helper function to format a 2D array into a string with row-wise separation.
    """
    return arr_sep.join([" ".join(map(str, row)) for row in array])

def labels_to_binary(label_list, input_labels):
    """
    Convert perceptions into a binary vector based on the label list.
    Handles both single strings and lists of strings for input_labels.
    """
    # Ensure input_labels is treated as a list
    if isinstance(input_labels, str):
        input_labels = [input_labels]
    
    # Create a set of lowercase input labels
    input_set = set(label.lower() for label in input_labels)
    
    # Generate the binary vector
    return [1 if label.lower() in input_set else 0 for label in label_list]



In [123]:
instruction0 = ""
instruction1 = "Classify the relationship between the input and output sequences based on perceptions"

# List
llama_data = prepare_data_for_multilabel_classification(dic_training,instruction0,label_list)

# Dict
llama_data_dict = {
    "instruction": [item["instruction"] for item in llama_data],
    "input": [item["input"] for item in llama_data],
    "output": [item["output"] for item in llama_data],
}

# Dataset
llama_data_dataset = Dataset.from_dict(llama_data_dict)

llama_data_dataset[0]


{'instruction': '',
 'input': 'I0 7 7\n7 7 7\n0 7 7\n---\nO0 0 0 0 7 7 0 7 7\n0 0 0 7 7 7 7 7 7\n0 0 0 0 7 7 0 7 7\n0 7 7 0 7 7 0 7 7\n7 7 7 7 7 7 7 7 7\n0 7 7 0 7 7 0 7 7\n0 0 0 0 7 7 0 7 7\n0 0 0 7 7 7 7 7 7\n0 0 0 0 7 7 0 7 7<EOS>\n---\nI4 0 4\n0 0 0\n0 4 0\n---\nO4 0 4 0 0 0 4 0 4\n0 0 0 0 0 0 0 0 0\n0 4 0 0 0 0 0 4 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 4 0 4 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 4 0 0 0 0<EOS>\n---\nI0 0 0\n0 0 2\n2 0 2\n---\nO0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 2\n0 0 0 0 0 0 2 0 2\n0 0 0 0 0 0 0 0 0\n0 0 2 0 0 0 0 0 2\n2 0 2 0 0 0 2 0 2<EOS>\n---\nI6 6 0\n6 0 0\n0 6 6\n---\nO6 6 0 6 6 0 0 0 0\n6 0 0 6 0 0 0 0 0\n0 6 6 0 6 6 0 0 0\n6 6 0 0 0 0 0 0 0\n6 0 0 0 0 0 0 0 0\n0 6 6 0 0 0 0 0 0\n0 0 0 6 6 0 6 6 0\n0 0 0 6 0 0 6 0 0\n0 0 0 0 6 6 0 6 6<EOS>\n---\nI2 2 2\n0 0 0\n0 2 2\n---\nO2 2 2 2 2 2 2 2 2\n0 0 0 0 0 0 0 0 0\n0 2 2 0 2 2 0 2 2\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 

### Load the models

In [ ]:
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig
from datasets import Dataset

# Load base model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "barc0/Llama-3.1-ARC-Potpourri-Transduction-8B"
)
#model.config.use_cache = False
#model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(
    "barc0/Llama-3.1-ARC-Potpourri-Transduction-8B"
)